# Retrieval API - Step 2 - Generate LLM (Gemini) Answer

## Code Initialization
Dependencies and environment initialization.

Ensure there's a `.env` file with your credentials in the same directory as this script. Use the `.env.example` file as template.

In [7]:
import os
from IPython.display import Markdown
import utils as u
from dotenv import load_dotenv

load_dotenv()

True

## Constants

In [8]:
GCP_PROJECT = os.getenv('GCP_PROJECT')
GCP_LOCATION = os.getenv('GCP_LOCATION')
GCP_MODEL = os.getenv('GCP_MODEL')

## Gemini Answer Generation

This is a downstream step that is only illustrative on how the response generation stage can be implemented. In the below example the retrieved articles are used to respond the prompt using an LLM hosted in **Google Cloud Vertex AI**. The passed request is built from the original prompt enhanced with the retrieved articles as gounding context.

The tested LLM is **Google Gemini 2.5 Flash Preview 05-20**. Response generations took a few seconds for a context between 5K to 8K LLM tokens.

### Prerequisite

To successfully complete the following steps, it's important that there's an active GCloud authentication in the environment where this notbeook is executed from. This can be done by running the following command:

```bash
$ gcloud auth application-default login
```

For more information see [Set up ADC for a local development environment](https://cloud.google.com/docs/authentication/set-up-adc-local-dev-environment).

### Read Chunks from [Step 1](1_get_chunks.ipynb)

In [9]:
frapi_query = u.load_dict_json('query')
chunks_list = u.load_dict_json('chunks')
frapi_prompt = frapi_query['data']['attributes']['query']['value']
display(Markdown(f"**Original Prompt:** {frapi_prompt}"))
display(Markdown(f"**Number of Chunks:** {len(chunks_list)}"))

**Original Prompt:** Summarise tariff decisions by the US administration over the last few months and their impact on the automotive industry worldwide

**Number of Chunks:** 10

### Gemini Structured Prompt

In [10]:
instructions_text ="""
    You are an experienced business analyst that respond in a professional manner.
    Answer the query using only the information provided in the list of articles.
    If you use information from an article, cite it using squared brackets containing the index number.
    At the end of the answer, show a list of the cited articles ordered by index and under the title Cited Articles.
    Each cited article must be displayed in the following Markdown format (don't forget the opening square bracket before the headline):
    - [index] [headline - source_name - publication_date](url)
    Use Markdown for the output.
"""
article_list = []

for chunk in chunks_list:
    article = {
        'index': len(article_list) + 1,
        'url': f"https://dj.factiva.com/article?id=drn:archive.newsarticle.{str(chunk['meta']['original_doc_id']).strip()}",
        'source_name': str(chunk['meta']['source']['name']).strip(),
        'headline': str(chunk['attributes']['headline']['main']['text']).strip(),
        'publication_date': chunk['attributes']['publication_date'],
        'content': f"{str(chunk['attributes']['snippet']['content'][0]['text']).strip()} {str(chunk['attributes']['content'][0]['text']).strip()}"
    }
    article_list.append(article)

llm_prompt = {
    'query': frapi_prompt,
    'articles': article_list,
    "instructions": instructions_text.strip()
}

In [11]:
# u.print_full_llm_prompt(llm_prompt)
u.print_partial_llm_prompt(llm_prompt)

{
    "query": "Summarise tariff decisions by the US administration over the last few months and their impact on the automotive industry worldwide",
    "articles": [
        {
            "index": 1,
            "url": "https://dj.factiva.com/article?id=drn:archive.newsarticle.DJDN000020250603el63002rw",
            "source_name": "Dow Jones Newswires",
            "headline": "Americans Still Hit the Car Lots in May -- WSJ",
            "publication_date": "2025-06-03",
            "content": "Americans are still buying a lot of cars, despite broader consumer concerns about the economy and rising prices as a result of tariffs. Americans Stil..."
        },
        {
            "index": 2,
            "url": "https://dj.factiva.com/article?id=drn:archive.newsarticle.RTCNWS0020250602el620008b",
            "source_name": "\u9053\u743c\u65af\u4e2d\u6587\u901a\u8baf (\u7b80\u4f53)",
            "headline": "\u7279\u6717\u666e\u8ba1\u5212\u4e0a\u8c03\u94a2\u94dd\u5173\u7a0e\u7684\u4e3e\u

### Gemini Request/Response

In [12]:
response = u.gemini_generate(llm_prompt, GCP_PROJECT, GCP_LOCATION, GCP_MODEL)
display(Markdown(f"## **Gemini Model:** {GCP_MODEL}"))
display(Markdown(f"## **Prompt:** {frapi_prompt}"))
display(Markdown(f"## **Gemini Response:**"))
display(Markdown(response))

## **Gemini Model:** gemini-2.5-flash-preview-05-20

## **Prompt:** Summarise tariff decisions by the US administration over the last few months and their impact on the automotive industry worldwide

## **Gemini Response:**

The US administration has recently implemented or threatened several tariff decisions impacting the automotive industry worldwide.

**Tariff Decisions by the US Administration:**
*   **Steel and Aluminum Tariffs:** President Trump plans to double existing tariffs on steel and aluminum to 50% from 25%, effective June 4, 2025 [2, 5]. These tariffs were initially applied on March 12, 2025 [5].
*   **General Automotive Import Tariffs:** There is an existing 25% import tariff on cars [6, 9]. President Trump has also threatened broader tariffs on automotive imports [1].
*   **Automotive Imports from India:** On March 26, 2025, the US adopted a measure to increase tariffs by 25% ad valorem on imports of passenger vehicles, light trucks, and certain automobile parts originating from India. This measure for auto parts became effective on May 3, 2025, and applies for an unlimited duration [3, 5, 8, 10].
*   **UK Trade Agreement:** A recent trade agreement with the US includes a 100,000-unit quota for a 10% tariff on car imports from the UK into the US [4].
*   **Domestic Parts Manufacturing:** President Trump expects automotive parts manufacturing to return to the US, giving the industry approximately one year to adjust its supply chains [6].

**Impact on the Automotive Industry Worldwide:**

*   **Increased Costs and Prices:**
    *   The planned doubling of steel and aluminum tariffs to 50% is expected to significantly increase manufacturing costs for carmakers in the US [2]. One estimate suggests this could add approximately 5% to cumulative Trump tariff costs, reaching over $7,000 per car [2].
    *   Toyota reported that US tariffs added $1.25 billion to its costs in April and May [1].
    *   Carmakers have begun raising prices to offset tariffs, particularly after existing pre-tariff inventory was sold [6].
    *   The push for domestic parts manufacturing poses a long-term challenge, as it is difficult to produce all parts in the US, especially lower-value components or those no longer manufactured domestically (e.g., LED displays). This could impact the affordability of new cars [6].

*   **Sales and Consumer Behavior:**
    *   Concerns about inflation and the impact of higher tariffs have weighed on consumer confidence, which remains near a record low [1].
    *   Despite low consumer confidence, Americans continue to purchase vehicles, with 1.2 million units sold in May, roughly in line with the previous year [1]. A rush of buying occurred in late March and April as consumers sought to get ahead of threatened automotive import tariffs [1].
    *   Americans spent around $53.8 billion on new vehicles in May, a 7% increase from a year prior, with an average price of $45,462 [1].
    *   Sales performance varied among manufacturers: Ford (up 16%), Toyota (up 11%), Honda (up 7%), Hyundai (up 8%), and Kia (up 5%) reported gains, while Mazda, which heavily relies on imports, saw its sales plunge 19% [1].
    *   In Korea, domestic sales for Hyundai Motor, Kia, KG Mobility, and GM Korea dropped in May, partly attributed to the US 25% auto tariff impacting exports [9].

*   **Financial Market Reactions:**
    *   Auto stocks, including Toyota, Stellantis, Ford, and General Motors, experienced declines globally following news of the planned steel and aluminum tariff increases, as investors worry about increased costs and the impact on demand [2, 6].

*   **International Trade Relations and WTO Challenges:**
    *   India has formally challenged the US 25% tariffs on passenger vehicles, light trucks, and auto components from India at the World Trade Organization (WTO) [3, 5, 8, 10]. India argues these are "safeguard measures" that the US failed to notify to the WTO and is seeking consultations and potential compensation [3, 5, 8, 10]. India's auto component exports to the US were estimated at $6.8 billion in FY24 [5].
    *   India had previously reserved the right to impose retaliatory duties on US products in response to steel and aluminum tariffs [3, 5].
    *   Korea's auto industry faces a "double whammy" of declining exports due to the US 25% auto tariff and a domestic slump. Abolishing this auto tariff is Korea's top priority in ongoing trade negotiations with the US [9].
    *   The UK's steel industry is experiencing a divergence in prices compared to the EU due to US tariffs, leading to an estimated £100-150 million annual revenue loss for UK companies like Tata Steel, indicating that the EU is moving faster to defend its industry against US tariffs [4].

*   **Supply Chain Adjustments:**
    *   Mexican companies are increasingly utilizing the Mexico-United States-Canada Agreement (T-MEC) to secure tariff-free access to the US market, particularly for trucks (nearly 100%), cars (80%), and auto parts (over 50%) [7].
    *   The T-MEC requires 75% regional content for automobiles to qualify for tariff-free access [7]. While some steel and aluminum products face 25% tariffs and cars 15% under T-MEC, the agreement generally provides preferential treatment compared to non-treaty exports, which would face higher tariffs (e.g., an average of 8.1% for Mexican automotive exports without T-MEC) [7].

**Cited Articles:**
*   [1] [Americans Still Hit the Car Lots in May -- WSJ - Dow Jones Newswires - 2025-06-03](https://dj.factiva.com/article?id=drn:archive.newsarticle.DJDN000020250603el63002rw)
*   [2] [特朗普计划上调钢铝关税的举动导致汽车股普遍走低 - 道琼斯中文通讯 (简体) - 2025-06-02](https://dj.factiva.com/article?id=drn:archive.newsarticle.RTCNWS0020250602el620008b)
*   [3] [India seeks WTO consultations with US over safeguard measures on auto components - Press Trust of India - 2025-06-03](https://dj.factiva.com/article?id=drn:archive.newsarticle.PRTRIN0020250603el6300c1g)
*   [4] [The recent trade agreement with the US "is the very best deal that the UK could... - Press Association National Newswire - 2025-06-03](https://dj.factiva.com/article?id=drn:archive.newsarticle.PRESSA0020250603el63006mx)
*   [5] [India seeks consultations with US at WTO on auto tariffs - BusinessLine Online - 2025-06-03](https://dj.factiva.com/article?id=drn:archive.newsarticle.BSNLNO0020250603el630010g)
*   [6] [Ford, GM Stocks Are Slipping. Investors Want a Brake on Tariffs. -- Barrons.com - Dow Jones Newswires - 2025-06-02](https://dj.factiva.com/article?id=drn:archive.newsarticle.DJDN000020250602el62002gf)
*   [7] [Automotive industry, the largest exporter within T-MEC - CE NAFTA 2.0-USMCA - 2025-06-02](https://dj.factiva.com/article?id=drn:archive.newsarticle.NAFTA00020250602el620002z)
*   [8] [India invokes WTO mechanism to address US safeguard duties on auto components - The Times of India - 2025-06-03](https://dj.factiva.com/article?id=drn:archive.newsarticle.TOI0000020250603el630008l)
*   [9] [Carmakers face double whammy of export fall, domestic slump amid leadership vacuum - Korea Times - 2025-06-03](https://dj.factiva.com/article?id=drn:archive.newsarticle.KORTIM0020250603el630000i)
*   [10] [Consultations Sought with US on Auto Parts Levies - The Economic Times - Delhi Edition - 2025-06-04](https://dj.factiva.com/article?id=drn:archive.newsarticle.ECTDEL0020250604el640000o)